<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%206/Modulo_6_Ejercicio_2_Transfer_Learning_CIFAR10_Resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ejercicio 2: Transfer Learning en Defectos Visuales de Producción con CIFAR-10**

*Aplicar técnicas de transferencia de aprendizaje usando modelos preentrenados (como MobileNetV2) para clasificar imágenes industriales.*

En este ejercicio aplicaremos una red neuronal convolucional preentrenada sobre el dataset CIFAR-10 para resolver una tarea de clasificación de imágenes.
Primero construiremos una CNN básica como baseline, y luego aplicaremos Transfer Learning usando MobileNetV2.
Evaluaremos el rendimiento con métricas de clasificación, curva de pérdida y matriz de confusión.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt


In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
# Seleccionar solo dos clases para simplificar (por ejemplo: avión y automóvil)
selected_classes = [0, 1]  # airplane vs automobile
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

X_train, y_train = X_train[train_mask], y_train[train_mask]
X_test, y_test = X_test[test_mask], y_test[test_mask]
y_train = (y_train == selected_classes[1]).astype(int).flatten()
y_test = (y_test == selected_classes[1]).astype(int).flatten()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 23s 0us/step


In [3]:
# Preprocesamiento para MobileNetV2
X_train_prep = preprocess_input(tf.image.resize(X_train, [96, 96]))
X_test_prep = preprocess_input(tf.image.resize(X_test, [96, 96]))

In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, validation_split=0.2, batch_size=64)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.6102 - loss: 8.6299 - val_accuracy: 0.8170 - val_loss: 0.4268
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8137 - loss: 0.4059 - val_accuracy: 0.8715 - val_loss: 0.2926
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8549 - loss: 0.3336 - val_accuracy: 0.8870 - val_loss: 0.2703
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8858 - loss: 0.2824 - val_accuracy: 0.8955 - val_loss: 0.2442
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8953 - loss: 0.2528 - val_accuracy: 0.9140 - val_loss: 0.2188


In [5]:
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      1000
           1       0.93      0.94      0.93      1000

    accuracy                           0.93      2000
   macro avg       0.93      0.93      0.93      2000
weighted avg       0.93      0.93      0.93      2000

[[928  72]
 [ 65 935]]


In [6]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
base_model.trainable = False

transfer_model = Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
transfer_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
transfer_model.fit(X_train_prep, y_train, epochs=5, validation_split=0.2, batch_size=64)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.9326 - loss: 0.1717 - val_accuracy: 0.9835 - val_loss: 0.0463
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9827 - loss: 0.0467 - val_accuracy: 0.9850 - val_loss: 0.0454
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.9876 - loss: 0.0366 - val_accuracy: 0.9835 - val_loss: 0.0451
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9886 - loss: 0.0300 - val_accuracy: 0.9840 - val_loss: 0.0492
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9919 - loss: 0.0240 - val_accuracy: 0.9860 - val_loss: 0.0411


In [7]:
y_pred = (transfer_model.predict(X_test_prep) > 0.5).astype(int)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1000
           1       0.99      0.99      0.99      1000

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000

[[985  15]
 [ 12 988]]


**Conclusiones:**

- El modelo preentrenado MobileNetV2 ha logrado buenos resultados con pocas épocas de entrenamiento.
- Comparado con la CNN básica, ofrece mayor generalización y menor overfitting.
- La transferencia de aprendizaje permite aprovechar conocimiento previo incluso en datasets pequeños.

**Próximos pasos:**
- Afinar los hiperparámetros y usar `model_checkpoint` y `early_stopping`.
- Descongelar capas del modelo base para realizar fine-tuning.
- Aplicar técnicas de aumento de datos para mejorar la robustez del modelo.